# Le plan: 

When one wants to get information 
## 1. Blabla intro du sujet TODO

## 2. What is our goal?

## 3. How do we achieve it (in short)?
~Use Quotebank + Unsupervised learning

## 4. Data story

## 5. Conclusions: 
### 5.1 What did we find?
### 5.2 What's next/would be interesting to do

### somewhere:
Indicate that we alternatively the terms speaker/citee/quotee, journal/media/newspaper, and group/consortium

# 1. Intro du sujet: TODO

# 2. Our goal:
We aim to show that we can recognize which media group a journal belongs to, using some automatic clustering algorithm using very little data. As a result, we will have shown a lack of diversity of these journals, thus showing that if you want to be well-informed, simply reading different newspapers is not enough. One needs to pay attention to more than that.

# 3. How we achieve that:

\[For a detailed description of the process, keep reading, or better, go see our [notebook](https://github.com/epfl-ada/ada-2021-project-mahj)\].


We need some *features* which allow us to distinguish between diferent media. To do so we use the [Quotebank](https://dlab.epfl.ch/people/west/pub/Vaucher-Spitz-Catasta-West_WSDM-21.pdf) database, which is a dataset of  speaker-attributed quotations that were extracted from diverse English news articles[^1].

[^1]: https://zenodo.org/record/4277311

We then augment this dataset using [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) to get the features we deemed necessary for our analysis.

**TODO: commenter la taille  É N O R M E  du dataset qu'on a?**  
**TODO: refine last paragraph**
After getting all those features, we use [Natural language processing](https://en.wikipedia.org/wiki/Natural_language_processing)-techniques (or NLP) to cluster media, and see what we find.
**TODO: expliquer pk on ne prend qu'une année. E.g. pcque les news changent bcp entre les années, esp with covid / different presidents, ...**

# 4. But more precisely, what did you do?:

### 4.1: data-extension
Well for starters, the data we extract from the Quotebank-dataset look like this:

In [3]:
# Ici quelques extraits de la data de quotebank avec juste les attributs qu'on a gardés

# Ou ptetre ?:
 # 1: original
 # 2: ce qu'on a gardé

As you can see, the owners of the media are not indicated here, so we need to extend our dataset with these features.  
If we can't automatically find a journal's parent group, we decide to throw it away.
We also do so if the journals are lonely children. Indeed, we seek (**todo: above all/firstly**) to cluster them according to their parent groups, so isolated media are irrelevant.

**TODO? "see file Wikidata_scraping.ipynb"**

In [7]:
# Excerpt of wikidata code for scraping?
# Exemple de à quoi ressemble les infos obtenues / la nouvelle data

### 4.2: Data processing?
Since for each we have both quotes, and quotees (i.e. the people quoted), it seems natural to try and cluster them according to these 2 features. Let us then try 2 different methods:
 * First, cluster according to which speakers are cited in it.
 * Second, cluster according to the text content of the extracted quotes.
 
**TODO: see file "Process_data.ipynb" for more details**
**TODO: do we speak about the distance metrics?**

In [9]:
# For 1, do a small graph like:
# journal i:
  # speaker 1
  # speaker 2
  # ...
  # speaker n

# For 2, do sth like:
# journal j:
  # quote 1
  # quote 2
  # ...
  # quote m

# Maybe 1 on the left, and 1 on the right.

For both these features, we will first use the [Term Frequency-Inverse Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) method (TF-IDF for short) (**todo: better name?**) before proceeding to further analysis.  
In a nutshell, this method allows one to measure the importance of a term[^1] to a document[^2] in a collection[^3] based on its number of apparitions.

[^1]: Here a citee, or a word
[^2]: Here a media
[^3]: Here, all the media we have kept

**TODO: maybe a bit long?**
Basically this method gives to each word a weight:
 * *proportional* to its number of occurences in d (term frequency)
 * and *inversely proportional* to its presence in the corpus (inverse document frequency)[^1]

The reasoning is that we use the **IDF** to **find** words **unique** to each document, or **remove recurrent** words, which we find in basically every document, and therefore don't bring any information (e.g. stopwords such as 'the' in a text).  
We use the **TF** to find words that **appear often**, as they may be **characteristic** of a particular document. For example the word 'brain' may often appear in a neuroscience paper, while not so much in a geography one.

[^1]: to quantify the 'presence' of a word in the corpus, we simply count the number of documents in which it occurs. We don't take its occurences per document into account here

In [14]:
# Illustrations: les formes des 2 matrices obtenues, peu importe les coefs

# Though on peut mettre une petit flèche vers
# la matrice journal-speaker[i, j] par ex.

### 4.3: Data analysis (the heart of our problem)
**TODO: see file "Analysis.ipynb" for more details**

Now that we have processed our data, we need to visualize it.  
The problem is that the data (our TF-IDF matrices) is inherently high-dimensional, so we as humans can't directly visualize it. That is why we transform our matrices using [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD).

This transformation should allow us to extract "[topics](https://en.wikipedia.org/wiki/Topic_model)", which correspond to the (orthonormal) directions of highest variation and their importance. We can then view the differences between different media along one such topic.

In [13]:
# Some illustration of SVD, or maybe of PCA to get the idea?

Let us apply this technique on the speakers matrix first.

In [17]:
# Illustration des projections qu'on a au départ, avec la séparation géographique.

Using the top speakers per axe, we identify the *topics* of the axes:

| Axis | Max                 | Min                     |
|------|---------------------|-------------------------|
| 0    | Frequent speakers   | Non Frequent speakers   |
| 1    | US speakers         | UK speakers             |
| 2    | Australian speakers | US speakers             |
| 3    | Canadian speakers   | US politician speakers  |
| 4    | Basketball speakers | US politicians speakers |

As you can see, the speakers are very different in newspapers from different countries which results in clustering mostly based on the country of origin of the newspapers.  
It makes sense: UK-media will cite more UK-speakers, while USA-media will cite more USA-speakers. 

So our clustering works (for example it is easy to recognize journals from the media group *News Corp Australia*), but it is not enough yet. 
Indeed, even if one is aware they should get their information from different media, are they really that likely to get it from different countries' media? Of course not, and that might be especially true for American people (**TODO: is there some study that says american are self centered? or at least we can find some about the nb of languages they know.**). 



Then let us see if we can differentiate the media *inside one country!*. We will focus on the USA since it is the country with the most data available

In [16]:
# show the list of plots "speaker with only USA"

Again we can try and extract the *topics* from the list of top speakers per axe:

| Axis | Max                             | Min                                              |
|------|---------------------------------|--------------------------------------------------|
| 0    | Frequent speakers               | Non Frequent speakers                          |
| 1    | Sport coaches of Los Angeles    | West coast personalities                       |
| 2    | Sport coaches of San Francisco  | Sport coaches/DJ of Los Angeles                |

We also see (in bright green) that journals from *Townsquare Media* explain most of the variance of several axes, namely the 3rd and the 4th.  
Hence although we can visually recognize media from this group, it might be interesting to redo our analysis while removing media from this group.